In [1]:
import tensorflow as tf

# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Available GPU(s): {gpus}")
else:
    print("No GPU found.")


Available GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import time
import os
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

In [2]:
def clean_directory(directory, extensions=['.jpg', '.jpeg', '.png']):
    for root, _, files in os.walk(directory):
        for file in files:
            if not any(file.lower().endswith(ext) for ext in extensions):
                os.remove(os.path.join(root, file))

clean_directory('DATA/Train')
clean_directory('DATA/Test')

In [3]:
ALLOWED_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}

def filter_images_only(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if not any(file.lower().endswith(ext) for ext in ALLOWED_EXTENSIONS):
                os.remove(os.path.join(root, file))
                print(f"Removed non-image file: {file}")

filter_images_only('DATA/Train')
filter_images_only('DATA/Test')


In [4]:
def count_images_in_class(directory):
    class_counts = {}
    for class_dir in os.listdir(directory):
        class_path = os.path.join(directory, class_dir)
        if os.path.isdir(class_path):
            num_images = len([f for f in os.listdir(class_path) if any(f.lower().endswith(ext) for ext in ALLOWED_EXTENSIONS)])
            class_counts[class_dir] = num_images
    return class_counts

train_class_counts = count_images_in_class('DATA/Train')
print("Training class counts:", train_class_counts)

total_images = sum(train_class_counts.values())
class_weights = {i: total_images / (2 * count) for i, count in enumerate(train_class_counts.values())}
print("Class weights:", class_weights)

Training class counts: {'Leaf': 12791, 'Not_Leaf': 12960}
Class weights: {0: 1.0066062074896411, 1: 0.993479938271605}


In [5]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='DATA/Train',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(256, 256)
)
validation_ds = keras.utils.image_dataset_from_directory(
    directory='DATA/Test',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(256, 256)
)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x91 in position 67: invalid start byte

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))


In [ ]:
for images, labels in train_ds.take(1):
    # Apply data augmentation
    augmented_images = data_augmentation(images, training=True)
    
    # Plot the augmented images
    plt.figure(figsize=(10, 10))
    for i in range(min(9, len(augmented_images))):  # Show up to 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.title("Augmented Image")
        plt.axis("off")
    plt.show()
    break  # Stop after taking one batch

In [8]:
l2_lambda = 0.0005

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu',
           input_shape=(256, 256, 3), kernel_regularizer=l2(l2_lambda)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'),

    Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(l2_lambda)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'),

    Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(l2_lambda)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(l2_lambda)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])
def lr_scheduler(epoch, lr):
    if epoch > 10:
        return lr * 0.5
    return lr

lr_scheduler_callback = LearningRateScheduler(lr_scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)



In [9]:
start_time = time.time()
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=100,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stopping, lr_scheduler_callback]
)
training_duration = time.time() - start_time

Epoch 1/100
805/805 [==============================] - 162s 189ms/step - loss: 1.6690 - accuracy: 0.8317 - precision: 0.8850 - recall: 0.7651 - val_loss: 0.7623 - val_accuracy: 0.9169 - val_precision: 0.8754 - val_recall: 0.9731 - lr: 0.0010
Epoch 2/100
805/805 [==============================] - 158s 196ms/step - loss: 0.7561 - accuracy: 0.8607 - precision: 0.8137 - recall: 0.9380 - val_loss: 0.4936 - val_accuracy: 0.9493 - val_precision: 0.9357 - val_recall: 0.9654 - lr: 0.0010
Epoch 3/100
805/805 [==============================] - 143s 178ms/step - loss: 0.4837 - accuracy: 0.9207 - precision: 0.8946 - recall: 0.9549 - val_loss: 0.5120 - val_accuracy: 0.9120 - val_precision: 0.8617 - val_recall: 0.9827 - lr: 0.0010
Epoch 4/100
805/805 [==============================] - 147s 182ms/step - loss: 0.3955 - accuracy: 0.9294 - precision: 0.9105 - recall: 0.9535 - val_loss: 0.4346 - val_accuracy: 0.9246 - val_precision: 0.8791 - val_recall: 0.9856 - lr: 0.0010
Epoch 5/100
805/805 [===========

In [10]:
model.save('new_model.h5')